In [14]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl
from itertools import islice

run_id = 181
qid = 1281

In [15]:
from openpyxl import Workbook, load_workbook

In [16]:
wb = load_workbook('../data/bibliometric nodes.xlsx')
ws = wb['useme']
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, index=idx, columns=cols)

In [18]:
df['id'] = df.index
df.head()

,label,timeset,weight<total_link_strength>,weight<norm._citations>,score<citations>,score<norm._citations>,score<pub._year>,weight<links>,weight<citations>,cluster,title,source,description,url,id
11,kamargianni (2013),None,162,1.3830,14,1.3830,2013,122,14,7,hybrid choice model to investigate effects of ...,transportation research record,"<table><tr><td>Authors:</td><td>kamargianni, m...",http://dx.doi.org/10.3141/2382-17,11
17,wang (2013),None,84,0.9878,10,0.9878,2013,61,10,3,policies and practices of low carbon city deve...,energy & environment,"<table><tr><td>Authors:</td><td>wang, c; lin, ...",None,17
23,shirmohammadi (2013),None,182,1.0866,11,1.0866,2013,118,11,1,analysis and design of tradable credit schemes...,transportation research record,"<table><tr><td>Authors:</td><td>shirmohammadi,...",http://dx.doi.org/10.3141/2333-04,23
28,jin (2013),None,168,0.4939,5,0.4939,2013,145,5,7,a recursive spatial equilibrium model for plan...,environment and planning b-planning & design,"<table><tr><td>Authors:</td><td>jin, y; echeni...",http://dx.doi.org/10.1068/b39134,28
43,schroeder (2013),None,80,0.8890,9,0.8890,2013,54,9,3,novel multisector networks and entrepreneurshi...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>schroeder, h; ...",http://dx.doi.org/10.1068/c3105ed,43


In [19]:
def get_topic(x):
    try:
        doi = x.replace('http://dx.doi.org/','')
        p = Doc.objects.get(wosarticle__di=doi)
        topic = DocTopic.objects.filter(run_id=run_id,doc=p).order_by('-score').first().topic.title
    except:
        topic = ""
    return(topic)

In [20]:
df['main_topic'] = df.apply(lambda x: get_topic(x['url']),axis=1)

In [13]:
df.to_csv('../data/bibliometric_nodes.csv',index=False)